In [ ]:
import pyo
from pyosc import WSine, WSaw, WSquare, WTri

In [ ]:
# Setup server
sr = 22050
s = pyo.Server(sr=sr)
s.boot()
s.start()

In [ ]:
notein = pyo.Notein(poly=1, scale=1)
freq = notein['pitch']

# Create mod amplitude control and selectors controls
mod_amp = pyo.Sig(0.1, mul=5000)
mod_sel_ctrl = pyo.Sig(0, mul=3)
osc_sel_ctrl = pyo.Sig(0, mul=3)

# Create 4 modulators and mix them in a selector
mod1 = pyo.Sine(freq) # Sine
mod2 = pyo.LFO(freq, type=3) # Triangle
mod3 = pyo.LFO(freq, type=2) # Square
mod4 = pyo.LFO(freq, type=1) # Saw
mod_sel = pyo.Selector([mod1, mod2, mod3, mod4], mod_sel_ctrl, mul=0.7)

# Create 4 carriers controlled by freq and the mods and mix them in a selector
osc1 = WSine(freq + mod_sel * mod_amp, samplerate=sr)
osc2 = WTri(freq + mod_sel * mod_amp, samplerate=sr)
osc3 = WSquare(freq + mod_sel * mod_amp, samplerate=sr)
osc4 = WSaw(freq + mod_sel * mod_amp, samplerate=sr)
osc_sel = pyo.Selector([osc1, osc2, osc3, osc4], osc_sel_ctrl, mul=0.7)

# Compress and output
comp = pyo.Tanh(osc_sel).out()

In [ ]:
# Utilities and controls
sc = pyo.Scope([comp, mod_sel])
sp = pyo.Spectrum(comp)
mod_amp.ctrl(title="Modulation amplitude")
mod_sel_ctrl.ctrl(title="Modulator selection")
osc_sel_ctrl.ctrl(title="Carrier selection")
notein.keyboard()

In [ ]:
# Launch gui
s.gui(locals())